# CREATE CALIBRATION

##### This first cell initializes the code. Under """Inputs""" you can adjust parameters if needed.

In [1]:
import os
import numpy as np
import tkinter as tk
from tkinter import filedialog
import image_handler as tif


if os.name == 'nt':
    win = True
else:
    win = False


default_data_path = f'{os.getcwd()}{os.sep}raw_data'
# Can put the filename here or run the next cell to override it.
cal_file = f'{default_data_path}{os.sep}SiBe_on_GIWAXmount_CenteredInBeam_flip.tif'


"""Inputs"""
detector = '"Eiger Stitch"'              #  Custom
calibrant = 'AgBh'                       #  Silver Behenate
wavelength = 1.54185                     #  angstroms
# mask_filename = None
mask_filename = 'beamstop_3stitch.edf'   #  File containing mask information
"""End Inputs"""


%gui qt

print('done')

done


### Find calibration image and flip it if needed

##### This cell is optional to find the tiff that you will be calibrating from. It could just be defined in the above cell manually.

In [2]:
"""Select file using GUI prompt (can uncomment filename above and skip)"""
root = tk.Tk()
if not win:
    root.withdraw()
    root.update()
cal_file = filedialog.askopenfilename(initialdir=default_data_path, title='Select Files to Flip', filetypes=(('TIFFs', '*.tif',), ('all', '*.*')))
root.destroy()
print(f'Using: {cal_file}\n    as calibration image')

"""If in the unflipped folder, we'll need to flip it"""
if f'{os.sep}unflipped{os.sep}' in cal_file.replace('/', os.sep):
    print('Flipping the image for pyFAI-calib2')
    # takes tif at 'filename' and rotates 180degrees and then saves in the folder 'save_path'
    # returns new filename of the flipped image
    cal_file = tif.flip(filename=cal_file, save_path=default_data_path)
    print(f'Flipped image location is\n    {cal_file}')
    
r_cal, c_cal = tif.size(cal_file)
print(f'The calibration image is {r_cal} x {c_cal} pixels')
# tif.view_tif(cal_file)

Using: /home/etortoric/Documents/GitHub/GIWAXS/raw_data/TT5mm-01-benzeneTPP_60min_flip.tif
    as calibration image
The calibration image is 1102 x 3070 pixels


### Run calibration GUI

##### Run this cell to open the PyFAI calibration GUI with above information preloaded in.

In [3]:
mask = f'{os.getcwd()}{os.sep}masks{os.sep}{mask_filename}'
string_to_send_system = f'pyFAI-calib2 {cal_file} -D {detector} -c {calibrant} -w {wavelength}'
if mask_filename:
    string_to_send_system += f' -m {mask}'
if not win:
    string_to_send_system += ' >/dev/null 2>&1'
os.system(string_to_send_system)
print('done')

done


##### Now the poni file is generated and GIWAXS_analyzer can be used.